# Clima en los cultivos

Uno de los factores principales a considerar durante las cosechas es el clima de la región, está determina las temporadas de cosecha y los tipos de cultivos que pueden existir. Por lo tanto será importante obtener la información metereológica del pais. 

## Extracción de datos

### Consistencia de la nueva información

Pretendemos obtener información del clima del país, sin embargo al observar la información obtenida del SIAP, vemos que la información esta dada según los municipios, por lo tanto debemos procurar que la información climática provenga de cada municipio. 

### Fuente
La información será extraída de 'Historical Weather API', que se puede encontrar en el siguiente elnace: https://open-meteo.com/en/docs/historical-weather-api#latitude=29.0881&longitude=-111.0103&start_date=2020-01-01&hourly=temperature_2m,relativehumidity_2m,precipitation. 

Lo principal de la API, es que proporciona información según las coordenadas geograficas de la región (latitud, longitud). Por lo tanto deberemos extraer las coordendadas de cada municipio.

### Proceso.

Esto lo haremos, mediante la extracción de información del siguiente enlace. 'https://www.coordenadas.com.es/mexico/index.php'. Para ello usaremos algunas librerías.

In [122]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import math
import re


Primero, obtendremos la lista de Estados y el número de municipios. Esto como referencia para más adelante.

In [123]:
url_ref = 'https://diaonia.com/cuantos-municipios-hay-en-cada-estado-de-la-republica-mexicana/'
estados_ref = pd.read_html(url_ref)
estados = pd.DataFrame(estados_ref[0])
estados.head()

,Estado,Cantidad de municipios
0,Aguascalientes,11
1,Baja California,5
2,Baja California Sur,5
3,Campeche,11
4,Chiapas,124


Ahora si, comenzamos con la extracción. Esto lo haremos mediante dos métodos.
#### Directo

El paso directo, consta en usar la 'coordenadas.com.es', una página que posee las coordendadas de muchas partes del mundo, entre ellas México, por lo tanto nos interesa. Si observa el enlace a continuación https://www.coordenadas.com.es/mexico/index.php, podrá ver que aparecen todos los estados de México, entonces la usaremos para automatizar la extracción de los municipios por estado.

In [124]:
coor_url = 'https://www.coordenadas.com.es/mexico/index.php'
html = requests.get(coor_url)
print(html)

<Response [200]>


Vemos que `requests` pudo conectar con la página. Entonces usaremos `BeautifulSoup` para revisar su contenido. 

In [125]:
html_content = html.text
soup = BeautifulSoup(html_content)

# Checando el código fuente de la página principal. Vemos que los enlaces que buscamos, se encuentra en cierta clase de HTML.
data = soup.find_all('a',class_= 'list-group-item')
url_estados = []
name_estados = []
for link in data:
    lhref = link.get('href')
    ltext = link.text
    url_estados.append(lhref)
    name_estados.append(ltext)

print(url_estados, name_estados)

i_cdmx = name_estados.index('Distrito-Federal')
name_estados[i_cdmx] = 'Ciudad de México'
name_estados.sort()

['https://www.coordenadas.com.es/mexico/pueblos-de-aguascalientes/1/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-baja-california/2/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-baja-california-sur/3/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-campeche/4/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-chiapas/5/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-chihuahua/6/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-coahuila-de-zaragoza/7/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-colima/8/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-distrito-federal/9/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-durango/10/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-guanajuato/11/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-guerrero/12/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-hidalgo/13/1', 'https://www.coordenadas.com.es/mexico/pueblos-de-jalisco/14/1', 'https://www.coordenadas.com.es/mexico/pueb

Entonces, podríamos generar un dataframe con los estados, su nombre clave y la url correspondiente.

In [126]:
estados['Nombre'] = name_estados
estados['Url'] = url_estados

estados.columns

Index(['Estado', 'Cantidad de municipios', 'Nombre', 'Url'], dtype='object')

Entonces a partir del enlace principal, obtuvimos los enlaces para obtener los enlances de los estados para obtener las coordenadas de los municipios. Ahora definimos un buscador mediante la clave para obtener la url, tomaremos como ejemplo a Sonora.

In [127]:
def info_df(estado,columna,df):
    i_e = list(df['Estado']).index(estado)
    if i_e >= 0:
        if columna == '*':
            data = df.loc[int(i_e),:]
        else:
            data = df.loc[int(i_e),columna]
    else: 
        data = 'No se encontro el estado'
    return data


info_df('Sonora','Url',estados)

'https://www.coordenadas.com.es/mexico/pueblos-de-sonora/26/1'

Ahora, obtenemos las coordenadas de los municipios.

In [128]:
turl = info_df('Sonora','Url',estados)
thtml = pd.read_html(turl)
tdf = pd.DataFrame(thtml[0])
tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ciudad       60 non-null     object
 1   Coordenadas  60 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


Obtuvimos las coordenadas de los municipios de Sonora, sin embargo, como se puede apreciar. Faltan municipios. 

In [129]:
print(f'Numero de renglones en Sonora: {tdf.shape[0]}\nNúmero de Municipios de Sonora: {info_df("Sonora","Cantidad de municipios",estados)}')

Numero de renglones en Sonora: 60
Número de Municipios de Sonora: 72


Revisando la página de los municipios de Sonora podemos ver que se encuentran desplegados en varias pestañas. Cada pestaña puede mostrar máximo 60 municipios. Entonces debemos encontrar los que faltan, La página mostrada esta representada por él último número del enlace. Entonces nos tocará modificarlo. Para ello considere la siguiente función.

In [130]:
def modify_tabpage(url,tab):
    url = url[:-1] + str(tab)
    return url

Ahora si podemos extraer todos los municipios de Sonora

In [131]:
n_s = tdf.shape[0]

n_url = modify_tabpage(turl, 2)

html_son = pd.read_html(n_url)
Sonora2 = pd.DataFrame(html_son[0])
S = pd.concat((tdf,Sonora2),ignore_index=True)
S.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ciudad       64 non-null     object
 1   Coordenadas  64 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


Pero aún así faltan municipios, este es el principal problema del primer método. Entonces vamos a concluir con la extracción de todas las coordendas DISPONIBLES en el url.

In [132]:
def simple_df(url):
    html_zero = pd.read_html(url)
    df = pd.DataFrame(html_zero[0])
    return df

def tabs_df(estado):
    n_muns = info_df(estado,'Cantidad de municipios',estados)
    url_zero = info_df(estado,'Url',estados)
    df = simple_df(url_zero)
    if n_muns > 60:
        max_tabs = math.ceil(n_muns / 60)
        for i in np.arange(2, max_tabs + 1):
            n_url = modify_tabpage(url_zero,i)
            n_df = simple_df(n_url)
            df = pd.concat((df, n_df),ignore_index= True)
    df['Estado'] = [estado for i in range(df.shape[0])]
    df = df[['Estado','Ciudad','Coordenadas']]
    return df

def mexico_muns_coords(selector):
    if selector == '*':
        selector = list(estados['Estado'])
    mex_df = pd.DataFrame()
    for estado in selector:
        est_df = tabs_df(estado)
        mex_df = pd.concat((mex_df,est_df),ignore_index=True)
    return mex_df

mmc = mexico_muns_coords(['Sonora','San Luis Potosí'])
mmc

,Estado,Ciudad,Coordenadas
0,Sonora,- Aconchi,"29.82504,-110.22717"
1,Sonora,- Agua-Prieta,"30.94849,-109.27437"
2,Sonora,- Alamos,"27.02694,-108.93659"
3,Sonora,- Altar,"30.71361,-111.83528"
4,Sonora,- Arivechi,"28.92799,-109.18830"
...,...,...,...
111,San Luis Potosí,- Villa-de-La-Paz,"23.68333,-100.71667"
112,San Luis Potosí,- Villa-de-Ramos,"22.99898,-102.17120"
113,San Luis Potosí,- Villa-de-Reyes,"21.77412,-101.01419"
114,San Luis Potosí,- Villa-Juarez,"22.32314,-100.26459"


Ahora, debemos darle formato. Usaremos la 'mmc' como base. 

In [133]:
mmc['Ciudad'] = mmc['Ciudad'].str.strip('-').str.strip().str.replace('-',' ')
mmc['Coordenadas'] = mmc['Coordenadas'].str.split(',')


Con el formato ya definido, lo anexamos a la función que genera la base de datos.

In [134]:
def mexico_muns_coords(selector):
    if selector == '*':
        selector = list(estados['Estado'])
    mex_df = pd.DataFrame()
    for estado in selector:
        est_df = tabs_df(estado)
        mex_df = pd.concat((mex_df,est_df),ignore_index=True)
    mex_df['Ciudad'] = mex_df['Ciudad'].str.strip('-').str.strip().str.replace('-',' ')
    mex_df['Coordenadas'] = mex_df['Coordenadas'].str.split(',')
    lat = []
    lon = []
    for coord in list(mex_df['Coordenadas']):
        lat.append(coord[0])
        lon.append(coord[1])
    mex_df['Latitud'] = lat
    mex_df['Longitud'] = lon
    mex_df.drop(columns='Coordenadas',inplace=True)
    return mex_df

mexico_muns_coords(['Sonora'])

,Estado,Ciudad,Latitud,Longitud
0,Sonora,Aconchi,29.82504,-110.22717
1,Sonora,Agua Prieta,30.94849,-109.27437
2,Sonora,Alamos,27.02694,-108.93659
3,Sonora,Altar,30.71361,-111.83528
4,Sonora,Arivechi,28.92799,-109.18830
...,...,...,...,...
59,Sonora,Trincheras,30.56667,-111.48333
60,Sonora,Tubutama,30.92182,-111.43449
61,Sonora,Ures,29.20698,-110.13310
62,Sonora,Villa Pesqueira,28.97680,-110.04024


Listo, obtuvimos las coordenadas de los municipios de sonora, en general de cualquier conjunto de estados. Sin embargo como vimos al comienzo nos faltaron municipios, entonces partiremos con el segundo método, un poco más complicado. Pero más completo*

#### Indirecto.

Para el método indirecto, nos basaremos en dos páginas.

* INEGI.
* CCA
* Wikipedia. 

Primero, observemos la url de la división municipal de Sonora. 'https://cuentame.inegi.org.mx/monografias/informacion/son/territorio/div_municipal.aspx', notemos que lo que va a cambiar entre estados es 'son' que es la abreviatura del estado. Entonces deberemos extraerlas, en eso entra CCA, en particula en el siguiente enlace 'http://www.cca.org.mx/ec/cursos/hb030/contenido/glosario_gentil.htm'. Una vez obtenidas extramos TODOS los municipios y usaremos las entradas de los municipios en Wikipedia para extraer las coordenadas. Entonces partimos con el primer paso.

##### Extraer Abreviaturas.

In [135]:
# Ya no funciona, el enlace se perdio. 
"""
url_abr = 'https://es.wikipedia.org/wiki/Plantilla:Abreviaciones_de_los_estados_de_M%C3%A9xico'
html_abr = pd.read_html(url_abr)
df_abr = html_abr[0][['Estado','Variable']]
df_abr = df_abr.drop(index = 32)
abr = list(df_abr['Variable'])
n_abr = []
for i in abr:
    if i.find(']') >0:
        i = i[0:(i.find(']')-2)]
    n_abr.append(i.lower().replace('.',"").replace(" ",""))
n_abr[abr.index('C.D.Mx.')] = 'df'
n_abr[14] = 'mex'

df_abr['Variable'] = n_abr
df_abr.head()
"""

'\nurl_abr = \'https://es.wikipedia.org/wiki/Plantilla:Abreviaciones_de_los_estados_de_M%C3%A9xico\'\nhtml_abr = pd.read_html(url_abr)\ndf_abr = html_abr[0][[\'Estado\',\'Variable\']]\ndf_abr = df_abr.drop(index = 32)\nabr = list(df_abr[\'Variable\'])\nn_abr = []\nfor i in abr:\n    if i.find(\']\') >0:\n        i = i[0:(i.find(\']\')-2)]\n    n_abr.append(i.lower().replace(\'.\',"").replace(" ",""))\nn_abr[abr.index(\'C.D.Mx.\')] = \'df\'\nn_abr[14] = \'mex\'\n\ndf_abr[\'Variable\'] = n_abr\ndf_abr.head()\n'

In [136]:
url_abr = 'http://www.cca.org.mx/ec/cursos/hb030/contenido/glosario_gentil.htm'
html_abr = pd.read_html(url_abr)
df_abr = pd.DataFrame(html_abr[1])[[0,1]]
df_abr.rename({0: 'Estado', 1: 'Clave'},axis= 1,inplace = True)
df_abr.drop(index = 0,inplace=True)
state = df_abr['Estado']
state = list(state)
code_re = []
for text in state:
    s = re.sub(r'\d{1,2}.','',text)
    s = s.lstrip()
    code_re.append(s)
df_abr['Estado'] = code_re
df_abr['Clave'] = df_abr['Clave'].str.lower().str.replace('é','e').str.replace('.','',regex=True)
df_abr.replace('Distrito Federal','Ciudad de México',inplace= True)
df_abr['n'] = [i for i in range(32)]
df_abr.set_index(['n'],inplace=True)
df_abr.head()

,Estado,Clave
n,,
0,Aguascalientes,ags
1,Baja California,bc
2,Baja California Sur,bcs
3,Campeche,camp
4,Coahuila,coah


##### Obtener Municipios

Ahora, iremos a INEGI a extraer la lista de municipos, de nuevo, experimentaremos con Sonora. 

In [137]:
inegi_state = 'https://cuentame.inegi.org.mx/monografias/informacion/son/territorio/div_municipal.aspx'
html_inegi = pd.read_html(inegi_state)
df_state = pd.DataFrame(html_inegi[3])[['Clave del municipio','Municipio']]
df_state.drop(df_state.index[-1],inplace=True)
df_state['Clave del municipio'] = df_state['Clave del municipio'].astype(int)
df_state.head()

,Clave del municipio,Municipio
0,1,Aconchi
1,2,Agua Prieta
2,3,Alamos
3,4,Altar
4,5,Arivechi


Una vez que funciona generamos una función similar a la del método directo. Para eso modificaremos una tabla anterior para facilitar el código.

In [138]:
estados['Url'] = df_abr['Clave']
estados.rename(columns = {'Url':'Abreviatura'},inplace=True)
estados.drop(columns='Nombre',inplace=True)

In [139]:
estados.head()

,Estado,Cantidad de municipios,Abreviatura
0,Aguascalientes,11,ags
1,Baja California,5,bc
2,Baja California Sur,5,bcs
3,Campeche,11,camp
4,Chiapas,124,coah


In [140]:
def df_muns_inegi(selector):
    burl_s = 'https://cuentame.inegi.org.mx/monografias/informacion/'
    burl_e = '/territorio/div_municipal.aspx'
    df = pd.DataFrame()
    if selector == '*':
        selector = list(estados['Estado'])
    for state in selector:
        a = info_df(state,'Abreviatura',estados)
        url_state = burl_s + a + burl_e
        html = pd.read_html(url_state,encoding='utf8')
        df_state = pd.DataFrame(html[3])[['Clave del municipio','Municipio']]
        df_state.drop(df_state.index[-1],inplace=True)
        df_state['Clave del municipio'] = df_state['Clave del municipio'].astype(int)
        df = pd.concat((df, df_state))
    return df

df_muns_inegi(['Aguascalientes'])           

,Clave del municipio,Municipio
0,1,Aguascalientes
1,2,Asientos
2,3,Calvillo
3,4,Cosío
4,5,Jesús María
5,6,Pabellón de Arteaga
6,7,Rincón de Romos
7,8,San José de Gracia
8,9,Tepezalá
9,10,El Llano


##### Obtener las coordenadas.
Finalmente obtendremos las coordenadas mediante Wikipedia. Ahora observe la extracción del municipio de Irapuato. Notemos que esto no es una tabla, es un valor dento del HTML. Entonces tocará hacerlo de la forma general, usando `requests` y `BeautifulSoup`. Veamos el siguiente ejemplo con Irapuato, Guanajuato.

In [141]:
url_coord = "https://es.wikipedia.org/wiki/Municipio_de_Irapuato"
 
r  = requests.get(url_coord).content
soup = BeautifulSoup(r,"lxml")

q = soup.find_all('span')
s_las = soup.select_one("span[class*= latitude]").text
s_la = s_las
s_lo = soup.select_one("span[class*= longitude]").text
print(s_la, s_lo)

20°41′00″N  101°22′00″O


Notemos que las coordenadas estan en la forma (grados, minutos, segundos, punto cardinal). Entonces, vamos a convertirlo a grados con decimales. Como detalle, la letra define el signo de la coordenada. 

* Positivo si es Norte y Oeste.
* Negativo si es Sur y Este.

Entonces debemos realizar la conversion.

In [142]:

u_coord = []
g_coord = ['°','′','″',' ']

a = s_la.split('°')
u_coord.append(a[0])
a = a[-1]
a = a.split('′')
u_coord.append(a[0])
a = a[-1]
a = a.split('″')
u_coord.append(a[0])
a = a[-1]
a = a.strip()
u_coord.append(a[0])

u = u_coord[:-1]
u_f = u_coord[-1]

for i in range(len(u)):
    u[i] = int(u[i])

u
s = 0
for i in range(len(u)):
    s += u[i] * 60 ** (-i)

if (u_f == 'N') | (u_f == 'W'):
    s = s
else:
    s = -s

u_coord, s

(['20', '41', '00', 'N'], 20.683333333333334)

Con esto, resumimos todo en una sola función.

In [143]:
### Funciones auxiliares
    # Extraer los valores de las coordenadas en grados, minutos, segundos y el cardinal.
def extract_coords_g(text):
    coord = []
    g_coord = ['°', '′', '″', 'sign']
    s = text
    for i in range(len(g_coord)):
            try:
                s.index(g_coord[i])
                s = s.split(g_coord[i])
                coord.append(s[0])
                s = s[-1]
            except:
                if i == (len(g_coord) - 1):
                     coord.append(s[0])
                else:
                    coord.append(str(0))
                continue
    return coord
    # Convertir los elementos de la lista en un tipo en particular. (En este caso entero)
def parse_list(g,t):
    g_t = []
    for i in g:
        g_t.append(t(i))
    return np.array(g_t)
    # Define el signo de la coordenadas.
def sign_coord(cor, g_f):

    if (g_f == 'N ') | (g_f == 'W '):
        cor = cor
    else:
        cor = -cor
    return cor
    # Conversion (g,m,s,c) a grados con decimal. 
def minsec_to_grades(coord):
    data = coord[:-1]
    data_f = coord[-1]
    cord = parse_list(data,int)
    c = 0
    for i,j in enumerate(cord):
        c+= j * 60 ** (-i)
    return sign_coord(c, data_f)


In [144]:
# Funcion final.
def coordinates(geo_info):
    coord = extract_coords_g(geo_info)
    return minsec_to_grades(coord)


s_la, coordinates(s_la)

('20°41′00″N ', -20.683333333333334)

Con esto logramos extraer los datos requeridos. Ahora debemos hacer la conexión de los 3 pasos del segundo método para tener lo que buscamos. Definiendo como valor de entrada el estado. 

In [145]:
# Definimos el o los estados
estado = 'Sonora'  
# Tomamos la lista de municipios.
e_muns = df_muns_inegi([estado])
e_muns.head()

url_base = "https://es.wikipedia.org/wiki/Municipio_de_"
municipies = []
for i in range(72):
    mun = e_muns['Municipio'].iloc[i]
    mun = mun.replace(' ','_')
    url_coord = url_base + mun
    try:    
        r  = requests.get(url_coord).content
        soup = BeautifulSoup(r,"lxml")
        q = soup.find_all('span')
        s_las = soup.select_one("span[class*= latitude]").text
        s_la = s_las
        s_lo = soup.select_one("span[class*= longitude]").text
        lat = coordinates(s_la)
        lon = coordinates(s_lo)
        municipies.append([estado, mun, lat, lon, 'Sucess'])
    except:
        municipies.append([estado, mun, np.nan, np.nan, 'Modificar URL'])

state_coords = pd.DataFrame(data = municipies, columns=['Estado','Municipio','Latitud','Longitud','Proceso'])



In [146]:
state_coords[state_coords['Proceso'] != 'Sucess']

,Estado,Municipio,Latitud,Longitud,Proceso
2,Sonora,Alamos,NaN,NaN,Modificar URL
35,Sonora,Magdalena,NaN,NaN,Modificar URL
36,Sonora,Mazatán,NaN,NaN,Modificar URL
37,Sonora,Moctezuma,NaN,NaN,Modificar URL
42,Sonora,Nogales,NaN,NaN,Modificar URL
49,Sonora,Rayón,NaN,NaN,Modificar URL
50,Sonora,Rosario,NaN,NaN,Modificar URL
70,Sonora,Benito_Juárez,NaN,NaN,Modificar URL


In [147]:
mun = e_muns['Municipio'].iloc[29]
mun = mun.replace(' ','_')
url_coord = url_base + mun
print(url_coord)

r  = requests.get(url_coord).content
soup = BeautifulSoup(r,"lxml")
q = soup.find_all('span')
s_las = soup.select_one("span[class*= latitude]").text
s_la = s_las
s_lo = soup.select_one("span[class*= longitude]").text

s_la, s_lo


def extract_coords_g(text):
    coord = []
    g_coord = ['°', '′', '″', 'sign']
    s = text
    for i in range(len(g_coord)):
            try:
                s.index(g_coord[i])
                s = s.split(g_coord[i])
                coord.append(s[0])
                s = s[-1]
            except:
                if i == (len(g_coord) - 1):
                     coord.append(s[0])
                else:
                    coord.append(str(0))
                continue
    return coord

extract_coords_g(s_la)


https://es.wikipedia.org/wiki/Municipio_de_Hermosillo


['29', '00', '0', 'N']

In [148]:
s_la
coord = []
g_coord = ['°', '′', '″', 'sign']
print(s_la.index(g_coord[0]))
s = s_la.split(g_coord[0])
s
coord.append(s[0])
s = s[-1]
print(s.index(g_coord[1]))
s = s.split(g_coord[1])
coord.append(s[0])
s = s[-1]
print(s.index)


2
2
<built-in method index of str object at 0x000002327FCC7370>
